In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD

import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.style.use('default')
# plt.style.use('dark_background')
%matplotlib inline

In [2]:
whiskey = pd.read_csv('./Scraping/Scraped_Data/master_whiskey.csv', encoding='latin1')

# drop f's and no grades
whiskey = whiskey[(whiskey['grade'] != 'f') & (whiskey['grade'] != 'No Grade')].copy()

# fill missing ages, vintages, region, abv
whiskey['age'].fillna('No Age', inplace=True)
whiskey['vint'].fillna('No Vint', inplace=True)
whiskey['region'].fillna('No Region', inplace=True)
whiskey['abv'].fillna('No Abv', inplace=True)

# drop "yrs" for the age
whiskey['age'] = whiskey['age'].map(lambda x : x.split(' yrs')[0])

# drop missing revies
whiskey = whiskey.dropna().copy()

# convert letter grades to numbers
grade_replacement = {'a': 5, 'aminus': 4.5, 'bplus': 4.25, 'b': 4, 'bminus': 3.5, 'cplus': 3.25, 'c': 3, 'cminus': 2.5, 'dplus': 2.25, 'd': 2, 'dminus': 1.5}
whiskey['grade'] = whiskey['grade'].map(grade_replacement)

In [3]:
beer = pd.read_csv('./Scraping/Scraped_Data/master_beer.csv', encoding='latin1')

# remove reviews without text reviews
beer = beer[beer['r_text'].str.contains('No Review') == False]
beer = beer[beer['breakdown'].str.contains('look:') == True]

# fill missing user name
beer['username'].fillna('Missing Username', inplace=True)
# fill nan ibu's
beer['ibu'].fillna('No IBU', inplace=True)

In [4]:
# create id's for modeling
whiskey['id'] = 'w'
beer['id'] = 'b'

In [5]:
stop = stopwords.words('english')

In [ ]:
# creating all_reviews df
beer.rename({'r_text':'review', 'score_y':'user_rating'}, axis=1, inplace=True)
whiskey.rename({'w_name':'name', 'grade':'user_rating', 'w_type':'style'}, axis=1, inplace=True)

all_reviews = beer[['name', 'review', 'username', 'user_rating', 'abv', 'style', 'id']].append(whiskey[['name', 'review', 'username', 'user_rating', 'abv', 'style', 'id']])

## Cleaning

In [5]:
beer.head()

,name,url,score_x,ratings,brewery,style,abv,ibu,brewer_notes,score_y,breakdown,username,r_text
0,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,American Double / Imperial Stout,8.3,60,Sep - Dec Brewed with flaked oats bitter...,4.82,look: 4.25 | smell: 4.75 | taste: 5 | feel: 4....,Nobel96,No Review
1,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,American Double / Imperial Stout,8.3,60,Sep - Dec Brewed with flaked oats bitter...,4.79,look: 5 | smell: 4.75 | taste: 4.75 | feel: 5 ...,Batman11,No Review
2,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,American Double / Imperial Stout,8.3,60,Sep - Dec Brewed with flaked oats bitter...,4.00,look: 4 | smell: 4 | taste: 4 | feel: 4 | ove...,Bloodsnot77,No Review
3,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,American Double / Imperial Stout,8.3,60,Sep - Dec Brewed with flaked oats bitter...,4.14,look: 4 | smell: 4.25 | taste: 4 | feel: 4.25 ...,jrc1093,No Review
4,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,American Double / Imperial Stout,8.3,60,Sep - Dec Brewed with flaked oats bitter...,4.09,look: 4.25 | smell: 4 | taste: 4 | feel: 4.25 ...,Maudrey,No Review


In [10]:
beer.shape

(630889, 13)

In [15]:
beer = beer[beer['r_text'].str.contains('No Review') == False]

In [16]:
beer.shape

(367279, 13)

In [20]:
beer[beer['breakdown'].str.contains('look:') != True]['name'].value_counts()

Raison D'Être                                    2175
OAKED Arrogant Bastard Ale                       2119
Samuel Adams Latitude 48 IPA                     2065
A Little Sumpin' Wild                            2031
Sublimely Self-Righteous Black IPA               1979
Red's Rye IPA                                    1957
Samuel Adams Cherry Wheat                        1914
Arctic Panzer Wolf                               1896
Samuel Adams Noble Pils                          1883
Stone Smoked Porter                              1877
Hop Head Red Ale                                 1864
Sierra Nevada Ruthless Rye IPA                   1842
Rogue Chocolate Stout                            1835
Northern Hemisphere Harvest Wet Hop IPA          1829
ApriHop                                          1812
Miles Davis' Bitches Brew                        1810
75 Minute IPA                                    1807
Sierra Nevada Porter                             1792
Hop Rod Rye                 

In [23]:
beer = beer[beer['breakdown'].str.contains('look:') == True]

In [24]:
beer.shape

(62487, 13)

In [25]:
beer.isnull().sum()

name              0
url               0
score_x           0
ratings           0
brewery           0
style             0
abv               0
ibu             470
brewer_notes      0
score_y           0
breakdown         0
username          1
r_text            0
dtype: int64

In [30]:
beer['username'].fillna('Missing Username', inplace=True)

In [36]:
beer['ibu'].value_counts()

No IBU    47649
60         2112
50         1674
70         1434
100         798
75          568
69          558
6           530
37          515
38          477
98          396
90          377
45          327
120         322
55          310
93          296
65          271
68          219
82          216
95          214
80          214
33          208
102         205
44          204
12          203
8           198
85          194
20          193
35          193
18          182
42          174
51          161
28          159
86          136
25          130
Name: ibu, dtype: int64

In [39]:
beer['ibu'].fillna('No IBU', inplace=True)

In [40]:
beer.isnull().sum()

name            0
url             0
score_x         0
ratings         0
brewery         0
style           0
abv             0
ibu             0
brewer_notes    0
score_y         0
breakdown       0
username        0
r_text          0
dtype: int64

In [41]:
whiskey.isnull().sum()

username      0
grade         0
review        6
w_name        0
bottler       0
age         125
w_type        0
vint        213
subtype       0
abv           2
region       47
price         0
avaib         0
url           0
dtype: int64

In [42]:
whiskey['age'].value_counts()

21 yrs     22
NAS yrs    21
16 yrs     18
8 yrs      17
19 yrs     17
18 yrs     17
12 yrs     16
7 yrs      14
14 yrs     14
25 yrs     12
15 yrs     12
10 yrs     11
5 yrs       9
23 yrs      9
20 yrs      8
26 yrs      8
17 yrs      7
2 yrs       6
13 yrs      6
24 yrs      6
30 yrs      6
3 yrs       5
36 yrs      5
29 yrs      5
33 yrs      5
50 yrs      5
11 yrs      4
27 yrs      3
37 yrs      3
40 yrs      2
31 yrs      2
22 yrs      2
35 yrs      2
9 yrs       1
6 yrs       1
4 yrs       1
32 yrs      1
Name: age, dtype: int64

In [45]:
whiskey[whiskey['age'].isnull()].head()

,username,grade,review,w_name,bottler,age,w_type,vint,subtype,abv,region,price,avaib,url
2,Andy,bplus,p: Candied plums and mint. The batches are of...,Aberlour A'bunadh Batch 46,OB,NaN,Scotch,NaN,Single Malt,60.4,Highland,65,Better Stores,http://www.lawhiskeysociety.com/whiskey-profil...
3,Fuji,bminus,Semi sweet chocolate and red berries. I get m...,Aberlour A'bunadh Batch 46,OB,NaN,Scotch,NaN,Single Malt,60.4,Highland,65,Better Stores,http://www.lawhiskeysociety.com/whiskey-profil...
4,Fuji,b,N: Ceral oats christmas cinnamon raisin raspbe...,Aberlour A'bunadh Batch 47,OB,NaN,Scotch,NaN,Single Malt,60.7,Speyside,65,Wide,http://www.lawhiskeysociety.com/whiskey-profil...
5,Andy,b,n: Cherry Kirsch in a metal thermos. p: Dipp...,Aberlour A'bunadh Batch 47,OB,NaN,Scotch,NaN,Single Malt,60.7,Speyside,65,Wide,http://www.lawhiskeysociety.com/whiskey-profil...
6,Andy,b,n: Plum pie whipped cream p: Hot and Christm...,Aberlour A'bunadh Batch 50,OB,NaN,Scotch,NaN,Single Malt,59.6,Highland,65,Better Stores,http://www.lawhiskeysociety.com/whiskey-profil...


In [46]:
whiskey['age'].fillna('No Age', inplace=True)

In [49]:
whiskey[whiskey['vint'].isnull()].head()

,username,grade,review,w_name,bottler,age,w_type,vint,subtype,abv,region,price,avaib,url
0,Ries,b,n: fuller rich nose; full vanilla caramels van...,Aberlour 12 Year Old Non Chill-filtered,Aberlour,12 yrs,Scotch,NaN,Single Malt,48.0,Speyside,50,Wide,http://www.lawhiskeysociety.com/whiskey-profil...
1,Andy,bminus,n: Gym socks across the room p: Wierd. Grea...,Aberlour 12 Year Old Non Chill-filtered,Aberlour,12 yrs,Scotch,NaN,Single Malt,48.0,Speyside,50,Wide,http://www.lawhiskeysociety.com/whiskey-profil...
2,Andy,bplus,p: Candied plums and mint. The batches are of...,Aberlour A'bunadh Batch 46,OB,No Age,Scotch,NaN,Single Malt,60.4,Highland,65,Better Stores,http://www.lawhiskeysociety.com/whiskey-profil...
3,Fuji,bminus,Semi sweet chocolate and red berries. I get m...,Aberlour A'bunadh Batch 46,OB,No Age,Scotch,NaN,Single Malt,60.4,Highland,65,Better Stores,http://www.lawhiskeysociety.com/whiskey-profil...
4,Fuji,b,N: Ceral oats christmas cinnamon raisin raspbe...,Aberlour A'bunadh Batch 47,OB,No Age,Scotch,NaN,Single Malt,60.7,Speyside,65,Wide,http://www.lawhiskeysociety.com/whiskey-profil...


In [50]:
whiskey['vint'].fillna('No Vint', inplace=True)

In [51]:
whiskey[whiskey['region'].isnull()].head()

,username,grade,review,w_name,bottler,age,w_type,vint,subtype,abv,region,price,avaib,url
8,Andy,b,n: Toasted oats and rose petals p: Cheerios....,Amrut Rye,OB,5 yrs,World,No Vint,Indian,50.0,NaN,180,Specialty Stores,http://www.lawhiskeysociety.com/whiskey-profil...
9,Sku,cminus,Nose: Grainy with vanilla notes.Palate: Raw an...,Amrut Rye,OB,5 yrs,World,No Vint,Indian,50.0,NaN,180,Specialty Stores,http://www.lawhiskeysociety.com/whiskey-profil...
48,Andy,cplus,n: Cheerios and cardboard. p: Neutral and bit...,Black Bull 21,Duncan Taylor,21 yrs,Scotch,No Vint,Blend,50.0,NaN,150,Better Stores,http://www.lawhiskeysociety.com/whiskey-profil...
49,Fuji,cplus,Semi sweet vanilla with fruit (apple and pear),Black Bull 21,Duncan Taylor,21 yrs,Scotch,No Vint,Blend,50.0,NaN,150,Better Stores,http://www.lawhiskeysociety.com/whiskey-profil...
74,Andy,cplus,n: A bit sour. Like a bad finish. p: Wood va...,Cadenhead Creations 20yr Batch 1,Cadenhead,20 yrs,Scotch,No Vint,Blend,46.0,NaN,99,Specialty Stores,http://www.lawhiskeysociety.com/whiskey-profil...


In [52]:
whiskey['region'].fillna('No Region', inplace=True)

In [63]:
whiskey['age'] = whiskey['age'].map(lambda x : x.split(' yrs')[0])

In [60]:
for i in whiskey['age']:
    i = i.split(' yrs')[0]
    print(i)

12
12
No Age
No Age
No Age
No Age
No Age
No Age
5
5
16
16
No Age
21
21
No Age
NAS
NAS
No Age
No Age
No Age
14
17
15
No Age
No Age
No Age
No Age
No Age
11
25
15
15
15
No Age
No Age
9
18
19
17
17
32
19
19
19
19
35
15
21
21
15
26
26
26
26
20
No Age
12
12
12
12
11
11
No Age
No Age
18
18
23
23
23
No Age
40
40
23
20
20
No Age
27
30
30
30
31
31
21
21
15
14
20
8
5
5
5
5
12
12
12
12
No Age
No Age
No Age
No Age
No Age
No Age
3
3
3
17
17
7
2
No Age
No Age
No Age
22
22
No Age
No Age
No Age
No Age
No Age
No Age
No Age
No Age
No Age
7
7
7
7
23
13
13
18
18
18
18
No Age
No Age
No Age
No Age
No Age
20
20
No Age
13
13
10
8
12
No Age
No Age
No Age
No Age
No Age
No Age
24
24
24
24
14
14
19
8
8
21
21
14
No Age
No Age
No Age
No Age
14
15
15
21
19
19
8
8
8
No Age
NAS
NAS
NAS
21
No Age
No Age
No Age
No Age
No Age
No Age
37
No Age
No Age
No Age
33
33
33
33
33
19
19
23
14
14
18
No Age
No Age
No Age
No Age
7
No Age
No Age
No Age
10
No Age
No Age
25
25
14
8
8
24
24
20
7
No Age
12
No Age
No Age
No Age
No Age
No Ag

In [64]:
whiskey['age'].value_counts()

No Age    125
21         22
NAS        21
16         18
8          17
18         17
19         17
12         16
14         14
7          14
25         12
15         12
10         11
23          9
5           9
20          8
26          8
17          7
13          6
2           6
24          6
30          6
29          5
50          5
33          5
3           5
36          5
11          4
27          3
37          3
40          2
22          2
31          2
35          2
32          1
4           1
9           1
6           1
Name: age, dtype: int64

In [85]:
whiskey.isnull().sum()

username    0
grade       0
review      6
w_name      0
bottler     0
age         0
w_type      0
vint        0
subtype     0
abv         0
region      0
price       0
avaib       0
url         0
dtype: int64

In [93]:
whiskey2.isnull().sum()

username    0
grade       0
review      0
w_name      0
bottler     0
age         0
w_type      0
vint        0
subtype     0
abv         0
region      0
price       0
avaib       0
url         0
dtype: int64

In [92]:
whiskey = whiskey.dropna().copy()

In [91]:
whiskey.shape

(428, 14)

In [86]:
whiskey['review'].dropna(inplace=True)

In [73]:
whiskey.shape

(428, 14)

In [114]:
whiskey.head()

,username,user_rating,review,name,bottler,age,style,vint,subtype,abv,region,price,avaib,url,id
0,Ries,4.00,n: fuller rich nose; full vanilla caramels van...,Aberlour 12 Year Old Non Chill-filtered,Aberlour,12,Scotch,No Vint,Single Malt,48,Speyside,50,Wide,http://www.lawhiskeysociety.com/whiskey-profil...,w
1,Andy,3.50,n: Gym socks across the room p: Wierd. Grea...,Aberlour 12 Year Old Non Chill-filtered,Aberlour,12,Scotch,No Vint,Single Malt,48,Speyside,50,Wide,http://www.lawhiskeysociety.com/whiskey-profil...,w
2,Andy,4.25,p: Candied plums and mint. The batches are of...,Aberlour A'bunadh Batch 46,OB,No Age,Scotch,No Vint,Single Malt,60.4,Highland,65,Better Stores,http://www.lawhiskeysociety.com/whiskey-profil...,w
3,Fuji,3.50,Semi sweet chocolate and red berries. I get m...,Aberlour A'bunadh Batch 46,OB,No Age,Scotch,No Vint,Single Malt,60.4,Highland,65,Better Stores,http://www.lawhiskeysociety.com/whiskey-profil...,w
4,Fuji,4.00,N: Ceral oats christmas cinnamon raisin raspbe...,Aberlour A'bunadh Batch 47,OB,No Age,Scotch,No Vint,Single Malt,60.7,Speyside,65,Wide,http://www.lawhiskeysociety.com/whiskey-profil...,w


In [115]:
beer.head()

,name,url,score_x,ratings,brewery,style,abv,ibu,brewer_notes,user_rating,breakdown,username,review,id
5,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,American Double / Imperial Stout,8.3,60,Sep - Dec Brewed with flaked oats bitter...,3.78,look: 4.5 | smell: 4.5 | taste: 3.5 | feel: 3....,ejmontoy1,Coffee forward beer that is good but not great...,b
7,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,American Double / Imperial Stout,8.3,60,Sep - Dec Brewed with flaked oats bitter...,4.41,look: 4.75 | smell: 4.25 | taste: 4.5 | feel: ...,lwillitz,poured from the tap of my kegerator into a por...,b
10,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,American Double / Imperial Stout,8.3,60,Sep - Dec Brewed with flaked oats bitter...,4.74,look: 4.5 | smell: 4.75 | taste: 4.75 | feel: ...,OldIrishHermit,"Look Founders sure craft some of the best ""loo...",b
11,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,American Double / Imperial Stout,8.3,60,Sep - Dec Brewed with flaked oats bitter...,4.33,look: 4 | smell: 4.75 | taste: 4.25 | feel: 4 ...,Taterturnt,An exceptional stout the flavor is balanced an...,b
18,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,American Double / Imperial Stout,8.3,60,Sep - Dec Brewed with flaked oats bitter...,4.70,look: 5 | smell: 5 | taste: 4.5 | feel: 4.5 | ...,smtilley,Its yummy and dark. I like my beers yummy and...,b


In [8]:
beer.rename({'r_text':'review', 'score_y':'user_rating'}, axis=1, inplace=True)

whiskey.rename({'w_name':'name', 'grade':'user_rating', 'w_type':'style'}, axis=1, inplace=True)

In [227]:
all_reviews[all_reviews['username'] == 'rodbeermunch']

,name,review,username,user_rating,abv,style,id
1189,Breakfast Stout,"Super dark brown and black with a 3/4"" beige h...",rodbeermunch,4.62,8.3,American Double / Imperial Stout,b
3275,90 Minute IPA,"Comes in that dark gold color 1/2"" white head ...",rodbeermunch,3.22,9,American Double / Imperial IPA,b
6221,Pliny The Elder,Appearance is a yellow color similar to macro ...,rodbeermunch,4.82,8,American Double / Imperial IPA,b
11780,Sculpin IPA,An oldie but a goodie. Light amber color soft...,rodbeermunch,4.30,7,American IPA,b
14749,Heady Topper,Gold color massive amounts of sediment through...,rodbeermunch,4.60,8,American Double / Imperial IPA,b
18906,KBS (Kentucky Breakfast Stout),Last time I had one of these was on Derby Day....,rodbeermunch,4.69,11.8,American Double / Imperial Stout,b
21093,Bourbon County Brand Stout,Been drinking these since 2008. Nice used mot...,rodbeermunch,4.49,14.1,American Double / Imperial Stout,b
23391,Old Rasputin,Pours an awesome and thick black beer with jus...,rodbeermunch,4.23,9,Russian Imperial Stout,b
25950,60 Minute IPA,Behold long rumored but often unverifiable the...,rodbeermunch,3.21,6,American IPA,b
36415,Enjoy By IPA,Substantial clarity looks professional as can ...,rodbeermunch,4.42,9.4,American Double / Imperial IPA,b


In [9]:
all_reviews = beer[['name', 'review', 'username', 'user_rating', 'abv', 'style', 'id']].append(whiskey[['name', 'review', 'username', 'user_rating', 'abv', 'style', 'id']])
all_reviews.head()

,name,review,username,user_rating,abv,style,id
5,Breakfast Stout,Coffee forward beer that is good but not great...,ejmontoy1,3.78,8.3,American Double / Imperial Stout,b
7,Breakfast Stout,poured from the tap of my kegerator into a por...,lwillitz,4.41,8.3,American Double / Imperial Stout,b
10,Breakfast Stout,"Look Founders sure craft some of the best ""loo...",OldIrishHermit,4.74,8.3,American Double / Imperial Stout,b
11,Breakfast Stout,An exceptional stout the flavor is balanced an...,Taterturnt,4.33,8.3,American Double / Imperial Stout,b
18,Breakfast Stout,Its yummy and dark. I like my beers yummy and...,smtilley,4.70,8.3,American Double / Imperial Stout,b


In [10]:
all_reviews['review'].isnull().sum()

0

In [11]:
all_reviews.shape

(62909, 7)

In [135]:
all_reviews = all_reviews.dropna().copy()

## Modeling

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
from sklearn.decomposition import TruncatedSVD

In [14]:
vect = CountVectorizer(ngram_range=(2,2), stop_words=stop, min_df=2)
X = vect.fit_transform(all_reviews['review'])

In [196]:
X.shape

(62909, 18642)

In [213]:
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)

In [214]:
X_svd = svd.fit_transform(X)

In [ ]:
# import pickle

# with open('C:/Users/Mark/Personal_GitHub/Portfolio/Capstone_What_Should_I_Drink/Pickle/svd.pkl', wb+) as f:
#     pickle.dump(X_svd, f)

In [222]:
X_svd.shape

(62909, 5)

In [220]:
cosine_similarity(X_svd)

MemoryError: 

In [221]:
import sys

sys.getsizeof(X_svd) / 1000000000

0.002516472

## Model with grouped reviews

In [22]:
grouped_reviews = all_reviews.groupby('name')['review'].sum()

In [25]:
grouped_reviews.head()

name
120 Minute IPA      This is kind of like two different beers.  Fre...
1554 Black Lager    L: very dark brown slightly clear one-finger h...
2XIPA               Had a pint of this on tap at 716 in Buffalo. N...
312 Urban Wheat     Fuzzy white foam that disappears fast. Leaves ...
60 Minute IPA       great ipa  its the one that got me 'hooked'One...
Name: review, dtype: object

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
from sklearn.decomposition import TruncatedSVD

In [26]:
vect = CountVectorizer(ngram_range=(2,2), stop_words=stop, min_df=2)
X = vect.fit_transform(grouped_reviews)

In [27]:
X.shape

(485, 264452)

In [79]:
svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)

In [80]:
X_svd = svd.fit_transform(X)

In [81]:
X_svd.shape

(485, 100)

In [ ]:
# import pickle

# with open('C:/Users/Mark/Personal_GitHub/Portfolio/Capstone_What_Should_I_Drink/Pickle/svd.pkl', wb+) as f:
#     pickle.dump(X_svd, f)

In [96]:
cos_sim = cosine_similarity(X_svd, X_svd)

In [97]:
cos_sim

array([[ 1.        ,  0.24986043,  0.44208858, ...,  0.38976273,
        -0.00803691,  0.47266724],
       [ 0.24986043,  1.        ,  0.32892689, ...,  0.26916323,
         0.09875946,  0.43421833],
       [ 0.44208858,  0.32892689,  1.        , ...,  0.50346862,
         0.0176143 ,  0.27989794],
       ...,
       [ 0.38976273,  0.26916323,  0.50346862, ...,  1.        ,
         0.03172681,  0.20464004],
       [-0.00803691,  0.09875946,  0.0176143 , ...,  0.03172681,
         1.        ,  0.08996898],
       [ 0.47266724,  0.43421833,  0.27989794, ...,  0.20464004,
         0.08996898,  1.        ]])

In [99]:
cos_sim.shape

(485, 485)

In [85]:
df_grouped_reviews = pd.DataFrame(grouped_reviews)
df_grouped_reviews.head()

,review
name,
120 Minute IPA,This is kind of like two different beers. Fre...
1554 Black Lager,L: very dark brown slightly clear one-finger h...
2XIPA,Had a pint of this on tap at 716 in Buffalo. N...
312 Urban Wheat,Fuzzy white foam that disappears fast. Leaves ...
60 Minute IPA,great ipa its the one that got me 'hooked'One...


In [53]:
df_grouped_reviews.index

Index(['120 Minute IPA', '1554 Black Lager', '2XIPA', '312 Urban Wheat',
       '60 Minute IPA', '75 Minute IPA', '90 Minute IPA',
       'A Little Sumpin' Sumpin' Ale', 'A Little Sumpin' Wild', 'Abbey Ale',
       ...
       'Willett Family Estate 25yr Barrel #2876  ', 'Wisconsin Belgian Red',
       'Wookey Jack', 'World Wide Stout', 'Yeti Imperial Stout',
       'Young's Double Chocolate Stout', 'Yuengling Traditional Lager',
       'Zombie Dust', 'anCnoc 16', '§ucaba'],
      dtype='object', name='name', length=485)

In [100]:
pd.DataFrame(cos_sim)

,0,1,2,3,4,5,6,7,8,9,...,475,476,477,478,479,480,481,482,483,484
0,1.000000,0.249860,0.442089,0.307070,0.551003,0.412091,0.636734,0.394489,0.389054,0.351131,...,-0.047142,0.214417,0.258510,0.410236,0.224020,0.206764,0.435696,0.389763,-0.008037,0.472667
1,0.249860,1.000000,0.328927,0.243651,0.335776,0.197242,0.330023,0.346326,0.337648,0.488373,...,0.194469,0.229560,0.581012,0.535921,0.519301,0.641802,0.362689,0.269163,0.098759,0.434218
2,0.442089,0.328927,1.000000,0.474397,0.728019,0.394136,0.737369,0.656666,0.585267,0.376499,...,0.021340,0.229191,0.353018,0.238111,0.232729,0.242328,0.537714,0.503469,0.017614,0.279898
3,0.307070,0.243651,0.474397,1.000000,0.465051,0.243711,0.388035,0.739674,0.437094,0.238790,...,0.061535,0.207171,0.179924,0.113276,0.129372,0.187261,0.474847,0.384858,0.031180,0.195824
4,0.551003,0.335776,0.728019,0.465051,1.000000,0.629268,0.920803,0.673022,0.551541,0.328781,...,0.041601,0.233314,0.349063,0.194499,0.178015,0.233863,0.555422,0.502957,0.015282,0.270862
5,0.412091,0.197242,0.394136,0.243711,0.629268,1.000000,0.600411,0.354530,0.361831,0.249814,...,0.089658,0.170226,0.219403,0.122474,0.134301,0.163418,0.283961,0.288810,-0.001898,0.179122
6,0.636734,0.330023,0.737369,0.388035,0.920803,0.600411,1.000000,0.642755,0.531854,0.388376,...,0.033589,0.241348,0.359193,0.246997,0.213373,0.233587,0.568891,0.500223,0.009787,0.331159
7,0.394489,0.346326,0.656666,0.739674,0.673022,0.354530,0.642755,1.000000,0.636593,0.328732,...,0.171995,0.236899,0.337622,0.176778,0.167603,0.211250,0.550742,0.582155,0.020625,0.252540
8,0.389054,0.337648,0.585267,0.437094,0.551541,0.361831,0.531854,0.636593,1.000000,0.436416,...,0.091040,0.214413,0.288206,0.157974,0.172829,0.193706,0.446661,0.456268,0.057757,0.247274
9,0.351131,0.488373,0.376499,0.238790,0.328781,0.249814,0.388376,0.328732,0.436416,1.000000,...,0.155281,0.249751,0.374533,0.510462,0.323967,0.309964,0.334909,0.273567,0.102697,0.577336


In [101]:
df_cos_sim = pd.DataFrame(cos_sim, index=df_grouped_reviews.index)
df_cos_sim

,0,1,2,3,4,5,6,7,8,9,...,475,476,477,478,479,480,481,482,483,484
name,,,,,,,,,,,,,,,,,,,,,
120 Minute IPA,1.000000,0.249860,0.442089,0.307070,0.551003,0.412091,0.636734,0.394489,0.389054,0.351131,...,-0.047142,0.214417,0.258510,0.410236,0.224020,0.206764,0.435696,0.389763,-0.008037,0.472667
1554 Black Lager,0.249860,1.000000,0.328927,0.243651,0.335776,0.197242,0.330023,0.346326,0.337648,0.488373,...,0.194469,0.229560,0.581012,0.535921,0.519301,0.641802,0.362689,0.269163,0.098759,0.434218
2XIPA,0.442089,0.328927,1.000000,0.474397,0.728019,0.394136,0.737369,0.656666,0.585267,0.376499,...,0.021340,0.229191,0.353018,0.238111,0.232729,0.242328,0.537714,0.503469,0.017614,0.279898
312 Urban Wheat,0.307070,0.243651,0.474397,1.000000,0.465051,0.243711,0.388035,0.739674,0.437094,0.238790,...,0.061535,0.207171,0.179924,0.113276,0.129372,0.187261,0.474847,0.384858,0.031180,0.195824
60 Minute IPA,0.551003,0.335776,0.728019,0.465051,1.000000,0.629268,0.920803,0.673022,0.551541,0.328781,...,0.041601,0.233314,0.349063,0.194499,0.178015,0.233863,0.555422,0.502957,0.015282,0.270862
75 Minute IPA,0.412091,0.197242,0.394136,0.243711,0.629268,1.000000,0.600411,0.354530,0.361831,0.249814,...,0.089658,0.170226,0.219403,0.122474,0.134301,0.163418,0.283961,0.288810,-0.001898,0.179122
90 Minute IPA,0.636734,0.330023,0.737369,0.388035,0.920803,0.600411,1.000000,0.642755,0.531854,0.388376,...,0.033589,0.241348,0.359193,0.246997,0.213373,0.233587,0.568891,0.500223,0.009787,0.331159
A Little Sumpin' Sumpin' Ale,0.394489,0.346326,0.656666,0.739674,0.673022,0.354530,0.642755,1.000000,0.636593,0.328732,...,0.171995,0.236899,0.337622,0.176778,0.167603,0.211250,0.550742,0.582155,0.020625,0.252540
A Little Sumpin' Wild,0.389054,0.337648,0.585267,0.437094,0.551541,0.361831,0.531854,0.636593,1.000000,0.436416,...,0.091040,0.214413,0.288206,0.157974,0.172829,0.193706,0.446661,0.456268,0.057757,0.247274


In [102]:
df_cos_reviews = pd.concat([df_grouped_reviews, df_cos_sim], axis=1)
df_cos_reviews.head()

,review,0,1,2,3,4,5,6,7,8,...,475,476,477,478,479,480,481,482,483,484
name,,,,,,,,,,,,,,,,,,,,,
120 Minute IPA,This is kind of like two different beers. Fre...,1.000000,0.249860,0.442089,0.307070,0.551003,0.412091,0.636734,0.394489,0.389054,...,-0.047142,0.214417,0.258510,0.410236,0.224020,0.206764,0.435696,0.389763,-0.008037,0.472667
1554 Black Lager,L: very dark brown slightly clear one-finger h...,0.249860,1.000000,0.328927,0.243651,0.335776,0.197242,0.330023,0.346326,0.337648,...,0.194469,0.229560,0.581012,0.535921,0.519301,0.641802,0.362689,0.269163,0.098759,0.434218
2XIPA,Had a pint of this on tap at 716 in Buffalo. N...,0.442089,0.328927,1.000000,0.474397,0.728019,0.394136,0.737369,0.656666,0.585267,...,0.021340,0.229191,0.353018,0.238111,0.232729,0.242328,0.537714,0.503469,0.017614,0.279898
312 Urban Wheat,Fuzzy white foam that disappears fast. Leaves ...,0.307070,0.243651,0.474397,1.000000,0.465051,0.243711,0.388035,0.739674,0.437094,...,0.061535,0.207171,0.179924,0.113276,0.129372,0.187261,0.474847,0.384858,0.031180,0.195824
60 Minute IPA,great ipa its the one that got me 'hooked'One...,0.551003,0.335776,0.728019,0.465051,1.000000,0.629268,0.920803,0.673022,0.551541,...,0.041601,0.233314,0.349063,0.194499,0.178015,0.233863,0.555422,0.502957,0.015282,0.270862


In [103]:
df_cos_reviews = df_cos_reviews.drop('review', axis=1)

In [104]:
df_cos_reviews.head()

,0,1,2,3,4,5,6,7,8,9,...,475,476,477,478,479,480,481,482,483,484
name,,,,,,,,,,,,,,,,,,,,,
120 Minute IPA,1.000000,0.249860,0.442089,0.307070,0.551003,0.412091,0.636734,0.394489,0.389054,0.351131,...,-0.047142,0.214417,0.258510,0.410236,0.224020,0.206764,0.435696,0.389763,-0.008037,0.472667
1554 Black Lager,0.249860,1.000000,0.328927,0.243651,0.335776,0.197242,0.330023,0.346326,0.337648,0.488373,...,0.194469,0.229560,0.581012,0.535921,0.519301,0.641802,0.362689,0.269163,0.098759,0.434218
2XIPA,0.442089,0.328927,1.000000,0.474397,0.728019,0.394136,0.737369,0.656666,0.585267,0.376499,...,0.021340,0.229191,0.353018,0.238111,0.232729,0.242328,0.537714,0.503469,0.017614,0.279898
312 Urban Wheat,0.307070,0.243651,0.474397,1.000000,0.465051,0.243711,0.388035,0.739674,0.437094,0.238790,...,0.061535,0.207171,0.179924,0.113276,0.129372,0.187261,0.474847,0.384858,0.031180,0.195824
60 Minute IPA,0.551003,0.335776,0.728019,0.465051,1.000000,0.629268,0.920803,0.673022,0.551541,0.328781,...,0.041601,0.233314,0.349063,0.194499,0.178015,0.233863,0.555422,0.502957,0.015282,0.270862


In [113]:
df_cos_reviews.columns = df_cos_reviews.index

In [114]:
df_cos_reviews

name,120 Minute IPA,1554 Black Lager,2XIPA,312 Urban Wheat,60 Minute IPA,75 Minute IPA,90 Minute IPA,A Little Sumpin' Sumpin' Ale,A Little Sumpin' Wild,Abbey Ale,...,Willett Family Estate 25yr Barrel #2876,Wisconsin Belgian Red,Wookey Jack,World Wide Stout,Yeti Imperial Stout,Young's Double Chocolate Stout,Yuengling Traditional Lager,Zombie Dust,anCnoc 16,§ucaba
name,,,,,,,,,,,,,,,,,,,,,
120 Minute IPA,1.000000,0.249860,0.442089,0.307070,0.551003,0.412091,0.636734,0.394489,0.389054,0.351131,...,-0.047142,0.214417,0.258510,0.410236,0.224020,0.206764,0.435696,0.389763,-0.008037,0.472667
1554 Black Lager,0.249860,1.000000,0.328927,0.243651,0.335776,0.197242,0.330023,0.346326,0.337648,0.488373,...,0.194469,0.229560,0.581012,0.535921,0.519301,0.641802,0.362689,0.269163,0.098759,0.434218
2XIPA,0.442089,0.328927,1.000000,0.474397,0.728019,0.394136,0.737369,0.656666,0.585267,0.376499,...,0.021340,0.229191,0.353018,0.238111,0.232729,0.242328,0.537714,0.503469,0.017614,0.279898
312 Urban Wheat,0.307070,0.243651,0.474397,1.000000,0.465051,0.243711,0.388035,0.739674,0.437094,0.238790,...,0.061535,0.207171,0.179924,0.113276,0.129372,0.187261,0.474847,0.384858,0.031180,0.195824
60 Minute IPA,0.551003,0.335776,0.728019,0.465051,1.000000,0.629268,0.920803,0.673022,0.551541,0.328781,...,0.041601,0.233314,0.349063,0.194499,0.178015,0.233863,0.555422,0.502957,0.015282,0.270862
75 Minute IPA,0.412091,0.197242,0.394136,0.243711,0.629268,1.000000,0.600411,0.354530,0.361831,0.249814,...,0.089658,0.170226,0.219403,0.122474,0.134301,0.163418,0.283961,0.288810,-0.001898,0.179122
90 Minute IPA,0.636734,0.330023,0.737369,0.388035,0.920803,0.600411,1.000000,0.642755,0.531854,0.388376,...,0.033589,0.241348,0.359193,0.246997,0.213373,0.233587,0.568891,0.500223,0.009787,0.331159
A Little Sumpin' Sumpin' Ale,0.394489,0.346326,0.656666,0.739674,0.673022,0.354530,0.642755,1.000000,0.636593,0.328732,...,0.171995,0.236899,0.337622,0.176778,0.167603,0.211250,0.550742,0.582155,0.020625,0.252540
A Little Sumpin' Wild,0.389054,0.337648,0.585267,0.437094,0.551541,0.361831,0.531854,0.636593,1.000000,0.436416,...,0.091040,0.214413,0.288206,0.157974,0.172829,0.193706,0.446661,0.456268,0.057757,0.247274


In [105]:
all_reviews.head()

,name,review,username,user_rating,abv,style,id
5,Breakfast Stout,Coffee forward beer that is good but not great...,ejmontoy1,3.78,8.3,American Double / Imperial Stout,b
7,Breakfast Stout,poured from the tap of my kegerator into a por...,lwillitz,4.41,8.3,American Double / Imperial Stout,b
10,Breakfast Stout,"Look Founders sure craft some of the best ""loo...",OldIrishHermit,4.74,8.3,American Double / Imperial Stout,b
11,Breakfast Stout,An exceptional stout the flavor is balanced an...,Taterturnt,4.33,8.3,American Double / Imperial Stout,b
18,Breakfast Stout,Its yummy and dark. I like my beers yummy and...,smtilley,4.70,8.3,American Double / Imperial Stout,b


In [115]:
all_reviews_cosine = all_reviews.merge(df_cos_reviews, left_on='name', right_index=True)

In [116]:
all_reviews_cosine.head()

,name,review,username,user_rating,abv,style,id,120 Minute IPA,1554 Black Lager,2XIPA,...,Willett Family Estate 25yr Barrel #2876,Wisconsin Belgian Red,Wookey Jack,World Wide Stout,Yeti Imperial Stout,Young's Double Chocolate Stout,Yuengling Traditional Lager,Zombie Dust,anCnoc 16,§ucaba
5,Breakfast Stout,Coffee forward beer that is good but not great...,ejmontoy1,3.78,8.3,American Double / Imperial Stout,b,0.195384,0.462178,0.204074,...,0.062362,0.12957,0.436429,0.506463,0.706302,0.663606,0.179465,0.157995,0.000664,0.304722
7,Breakfast Stout,poured from the tap of my kegerator into a por...,lwillitz,4.41,8.3,American Double / Imperial Stout,b,0.195384,0.462178,0.204074,...,0.062362,0.12957,0.436429,0.506463,0.706302,0.663606,0.179465,0.157995,0.000664,0.304722
10,Breakfast Stout,"Look Founders sure craft some of the best ""loo...",OldIrishHermit,4.74,8.3,American Double / Imperial Stout,b,0.195384,0.462178,0.204074,...,0.062362,0.12957,0.436429,0.506463,0.706302,0.663606,0.179465,0.157995,0.000664,0.304722
11,Breakfast Stout,An exceptional stout the flavor is balanced an...,Taterturnt,4.33,8.3,American Double / Imperial Stout,b,0.195384,0.462178,0.204074,...,0.062362,0.12957,0.436429,0.506463,0.706302,0.663606,0.179465,0.157995,0.000664,0.304722
18,Breakfast Stout,Its yummy and dark. I like my beers yummy and...,smtilley,4.70,8.3,American Double / Imperial Stout,b,0.195384,0.462178,0.204074,...,0.062362,0.12957,0.436429,0.506463,0.706302,0.663606,0.179465,0.157995,0.000664,0.304722


In [117]:
all_reviews_cosine.to_csv('./Scraping/Scraped_Data/Data/all_reviews_cosine.csv', index=False)

## Recommender

In [120]:
all_reviews_cosine = pd.read_csv('./Scraping/Scraped_Data/Data/all_reviews_cosine.csv')

C:\Users\Mark\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [124]:
beer.head()

,name,url,score_x,ratings,brewery,style,abv,ibu,brewer_notes,user_rating,breakdown,username,review,id
5,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,American Double / Imperial Stout,8.3,60,Sep - Dec Brewed with flaked oats bitter...,3.78,look: 4.5 | smell: 4.5 | taste: 3.5 | feel: 3....,ejmontoy1,Coffee forward beer that is good but not great...,b
7,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,American Double / Imperial Stout,8.3,60,Sep - Dec Brewed with flaked oats bitter...,4.41,look: 4.75 | smell: 4.25 | taste: 4.5 | feel: ...,lwillitz,poured from the tap of my kegerator into a por...,b
10,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,American Double / Imperial Stout,8.3,60,Sep - Dec Brewed with flaked oats bitter...,4.74,look: 4.5 | smell: 4.75 | taste: 4.75 | feel: ...,OldIrishHermit,"Look Founders sure craft some of the best ""loo...",b
11,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,American Double / Imperial Stout,8.3,60,Sep - Dec Brewed with flaked oats bitter...,4.33,look: 4 | smell: 4.75 | taste: 4.25 | feel: 4 ...,Taterturnt,An exceptional stout the flavor is balanced an...,b
18,Breakfast Stout,/beer/profile/1199/11757/,4.51,"16,948",Founders Brewing Company,American Double / Imperial Stout,8.3,60,Sep - Dec Brewed with flaked oats bitter...,4.70,look: 5 | smell: 5 | taste: 4.5 | feel: 4.5 | ...,smtilley,Its yummy and dark. I like my beers yummy and...,b


In [125]:
whiskey.head()

,username,user_rating,review,name,bottler,age,style,vint,subtype,abv,region,price,avaib,url,id
0,Ries,4.00,n: fuller rich nose; full vanilla caramels van...,Aberlour 12 Year Old Non Chill-filtered,Aberlour,12,Scotch,No Vint,Single Malt,48,Speyside,50,Wide,http://www.lawhiskeysociety.com/whiskey-profil...,w
1,Andy,3.50,n: Gym socks across the room p: Wierd. Grea...,Aberlour 12 Year Old Non Chill-filtered,Aberlour,12,Scotch,No Vint,Single Malt,48,Speyside,50,Wide,http://www.lawhiskeysociety.com/whiskey-profil...,w
2,Andy,4.25,p: Candied plums and mint. The batches are of...,Aberlour A'bunadh Batch 46,OB,No Age,Scotch,No Vint,Single Malt,60.4,Highland,65,Better Stores,http://www.lawhiskeysociety.com/whiskey-profil...,w
3,Fuji,3.50,Semi sweet chocolate and red berries. I get m...,Aberlour A'bunadh Batch 46,OB,No Age,Scotch,No Vint,Single Malt,60.4,Highland,65,Better Stores,http://www.lawhiskeysociety.com/whiskey-profil...,w
4,Fuji,4.00,N: Ceral oats christmas cinnamon raisin raspbe...,Aberlour A'bunadh Batch 47,OB,No Age,Scotch,No Vint,Single Malt,60.7,Speyside,65,Wide,http://www.lawhiskeysociety.com/whiskey-profil...,w


In [121]:
all_reviews_cosine

,name,review,username,user_rating,abv,style,id,120 Minute IPA,1554 Black Lager,2XIPA,...,Willett Family Estate 25yr Barrel #2876,Wisconsin Belgian Red,Wookey Jack,World Wide Stout,Yeti Imperial Stout,Young's Double Chocolate Stout,Yuengling Traditional Lager,Zombie Dust,anCnoc 16,§ucaba
0,Breakfast Stout,Coffee forward beer that is good but not great...,ejmontoy1,3.78,8.3,American Double / Imperial Stout,b,0.195384,0.462178,0.204074,...,0.062362,0.129570,0.436429,0.506463,0.706302,0.663606,0.179465,0.157995,0.000664,0.304722
1,Breakfast Stout,poured from the tap of my kegerator into a por...,lwillitz,4.41,8.3,American Double / Imperial Stout,b,0.195384,0.462178,0.204074,...,0.062362,0.129570,0.436429,0.506463,0.706302,0.663606,0.179465,0.157995,0.000664,0.304722
2,Breakfast Stout,"Look Founders sure craft some of the best ""loo...",OldIrishHermit,4.74,8.3,American Double / Imperial Stout,b,0.195384,0.462178,0.204074,...,0.062362,0.129570,0.436429,0.506463,0.706302,0.663606,0.179465,0.157995,0.000664,0.304722
3,Breakfast Stout,An exceptional stout the flavor is balanced an...,Taterturnt,4.33,8.3,American Double / Imperial Stout,b,0.195384,0.462178,0.204074,...,0.062362,0.129570,0.436429,0.506463,0.706302,0.663606,0.179465,0.157995,0.000664,0.304722
4,Breakfast Stout,Its yummy and dark. I like my beers yummy and...,smtilley,4.70,8.3,American Double / Imperial Stout,b,0.195384,0.462178,0.204074,...,0.062362,0.129570,0.436429,0.506463,0.706302,0.663606,0.179465,0.157995,0.000664,0.304722
5,Breakfast Stout,A meal by its self!,MonkeyLord,4.64,8.3,American Double / Imperial Stout,b,0.195384,0.462178,0.204074,...,0.062362,0.129570,0.436429,0.506463,0.706302,0.663606,0.179465,0.157995,0.000664,0.304722
6,Breakfast Stout,Dark the Coffee and chocolate jump out mildly ...,Almighty1,4.37,8.3,American Double / Imperial Stout,b,0.195384,0.462178,0.204074,...,0.062362,0.129570,0.436429,0.506463,0.706302,0.663606,0.179465,0.157995,0.000664,0.304722
7,Breakfast Stout,Nope. Not for me. Just too mealy. I prefer the...,Jesterr44,3.73,8.3,American Double / Imperial Stout,b,0.195384,0.462178,0.204074,...,0.062362,0.129570,0.436429,0.506463,0.706302,0.663606,0.179465,0.157995,0.000664,0.304722
8,Breakfast Stout,Breakfast YUM! This beer pours a dark mahogany...,Fading2black,4.29,8.3,American Double / Imperial Stout,b,0.195384,0.462178,0.204074,...,0.062362,0.129570,0.436429,0.506463,0.706302,0.663606,0.179465,0.157995,0.000664,0.304722
9,Breakfast Stout,Draft at the Industry. Dark brown milky tight...,stevoj,4.36,8.3,American Double / Imperial Stout,b,0.195384,0.462178,0.204074,...,0.062362,0.129570,0.436429,0.506463,0.706302,0.663606,0.179465,0.157995,0.000664,0.304722
